**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install opencv-python scikit-video

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization


import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

     |████████████████████████████████| 2.3MB 3.5MB/s 


Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



['/job:localhost/replica:0/task:0/device:GPU:0']

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

___
**Answer 1**:
This function returns the next action to do, given a state $s$. It is possible to decide whether or not to face the exploration-exploitation dilemma via the ``train`` parameter.
When ``train=True``, it randomly samples a number uniformly between 0 and 1 and depending on the value of this number, we decide to explore or exploit. Here is when $\epsilon$ is essential.

$\epsilon$ decides whether we stuck in our learned startegy (smaller values of $\epsilon$) or we explore the environment by following random actions (higher value of $\epsilon$).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=21 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

____
__Answer 2__:
The environment is represented as a grid and ``position`` and ``board`` are the representations of this grid. The difference between these two representations of the same things is in the information the contain. ``postition`` represents the cells the agent can go (it can't go to the cells labelled as -1 in ``position``). ``board`` contains the rewards still available (if the agent has already gone to a cell, there would be a 0 in that cell in ``board``.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        game_over = False
        win = 0
        lose = 0
        state = env.reset()

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/11.0. Average score (-2.0)
Win/lose count 6.5/12.0. Average score (-3.75)
Win/lose count 12.5/14.0. Average score (-3.0)
Win/lose count 7.5/14.0. Average score (-3.875)
Win/lose count 17.0/16.0. Average score (-2.9)
Win/lose count 11.0/14.0. Average score (-2.9166666666666665)
Win/lose count 9.5/15.0. Average score (-3.2857142857142856)
Win/lose count 9.5/18.0. Average score (-3.9375)
Win/lose count 9.5/11.0. Average score (-3.6666666666666665)
Win/lose count 13.5/20.0. Average score (-3.95)
Win/lose count 10.5/16.0. Average score (-4.090909090909091)
Win/lose count 10.0/6.0. Average score (-3.4166666666666665)
Win/lose count 8.0/11.0. Average score (-3.3846153846153846)
Win/lose count 12.0/12.0. Average score (-3.142857142857143)
Win/lose count 12.0/14.0. Average score (-3.066666666666667)
Final score: -3.066666666666667


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




$\bullet$
From the Bellman Equation,

\begin{align}
Q^\pi(s,a) & = E_{p^{\pi}}\left[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \\
 & = E_{p^{\pi}}\left[\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t}) + r(s_0,a_0)|s_{0}=s,a_{0}=a\right] \\
 & = E_{p^{\pi}}\left[\gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t}) + r(s_0,a_0)|s_{0}=s,a_{0}=a\right] \\
  & = r(s,a) + \gamma E_{p^{\pi}}\left[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1}) |s_{0}=s,a_{0}=a\right]
\end{align}

But we have:

\begin{align}
E_{p^{\pi}}\left[\sum_{t \geq 0}\gamma^{t}r(s_{t+1},a_{t+1}) |s_{0}=s,a_{0}=a\right] & = \sum_{a',s'} p^{\pi}(s_1 = s',a_1 = a') E_{p^{\pi}}\left[\sum_{t \geq 0}\gamma^{t}r(s_{t+1},a_{t+1}) |s_{1}=s',a_{1}=a'\right] \\
 & = \sum_{a',s'} p^{\pi}(s_1 = s',a_1 = a')Q^{\pi}(s';a')
\end{align}

Thus,
\begin{equation}
Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation}


*******************
*******************


$\bullet$
Optimal policy:

\begin{align}
Q^*(s,a) & = \underset{\pi}{\text{max}} \;Q^{\pi}(s,a) \\
 & = \underset{\pi}{\text{max}} \; E_{(s',a')\sim \pi(.|s,a)}\left[r(s,a)+\gamma \, Q^{\pi}(s',a')\right] \\
 & = E_{(s',a')\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma \underset{\pi}{\text{ max}} \, Q^{\pi}(s',a')\right] \\
\end{align}

To maximize with respect to the policy we have to chose the best action:
\begin{align}
 & = E_{s'\sim \pi(.|s,a)}\left[r(s,a)+\gamma \underset{\pi, a' }{\text{ max}} \;  Q^{\pi}(s',a')\right] \\
& = \underset{\pi }{\text{max}} \; E_{s'\sim \pi(.|s,a)}\left[r(s,a)+\gamma \underset{a'}{\text{ max}} \; Q^{\pi}(s',a')\right] \\
& =  E_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma \underset{a'}{\text{ max}} \; Q^*(s',a')\right]
\end{align}

***************
***************

$\bullet$
We showed that the optimal $Q$ function is the fixed point of the optimal Bellman operator. Thus, it is a good choice of loss to reduce the distance between the two terms : $Q^{\pi}(s,a,\theta)$ and $E_{s' \sim \pi^*(.|s,a)}\left[ r+\gamma\, \underset{a'}{\text{ max }} Q^{\pi}(s',a,\theta)\right]$. So,

\begin{equation}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\left\| r+\gamma\underset{a'}{\text{ max }}Q(s',a,\theta)-Q(s,a,\theta)\right\|^{2}
\end{equation}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<self.max_memory:
            self.memory.append(m)

    def random_access(self):
        if len(self.memory) == 0:
            return None
        else:
            return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s,axis=0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, n_s , a , r , game_over = self.memory.random_access()
            target_q[i] = self.model.predict(np.expand_dims(s,axis=0))
            input_states[i] = s
            
            if game_over:
                target_q[i,a] = r
            else:
                target_q[i][a] = r + self.discount * np.max(self.model.predict(np.expand_dims(n_s,0)))

        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential([Flatten(),
                            Dense(16, activation='relu'),
                            Dense(4)
                            ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

Epoch 000/021 | Loss 0.0024 | Win/lose count 2.0/6.0 (-4.0)
Epoch 001/021 | Loss 0.0036 | Win/lose count 2.5/7.0 (-4.5)
Epoch 002/021 | Loss 0.0037 | Win/lose count 7.5/5.0 (2.5)
Epoch 003/021 | Loss 0.0165 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/021 | Loss 0.0332 | Win/lose count 1.5/3.0 (-1.5)
Epoch 005/021 | Loss 0.0064 | Win/lose count 1.0/5.0 (-4.0)
Epoch 006/021 | Loss 0.0102 | Win/lose count 2.5/5.0 (-2.5)
Epoch 007/021 | Loss 0.0136 | Win/lose count 4.0/3.0 (1.0)
Epoch 008/021 | Loss 0.0035 | Win/lose count 2.0/1.0 (1.0)
Epoch 009/021 | Loss 0.0120 | Win/lose count 1.0/0 (1.0)
Epoch 010/021 | Loss 0.0033 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/021 | Loss 0.0064 | Win/lose count 1.5/3.0 (-1.5)
Epoch 012/021 | Loss 0.0033 | Win/lose count 3.5/4.0 (-0.5)
Epoch 013/021 | Loss 0.0040 | Win/lose count 3.0/3.0 (0.0)
Epoch 014/021 | Loss 0.0091 | Win/lose count 4.5/5.0 (-0.5)
Epoch 015/021 | Loss 0.0037 | Win/lose count 8.0/3.0 (5.0)
Epoch 016/021 | Loss 0.0042 | Win/lose count 4.0/2

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([Conv2D(16, (2,2), activation='relu'),
                            Conv2D(32, (2,2), activation='relu'),
                            Flatten(),
                            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

Epoch 000/021 | Loss 0.0010 | Win/lose count 4.0/2.0 (2.0)
Epoch 001/021 | Loss 0.0043 | Win/lose count 2.5/5.0 (-2.5)
Epoch 002/021 | Loss 0.0072 | Win/lose count 6.0/3.0 (3.0)
Epoch 003/021 | Loss 0.0032 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/021 | Loss 0.0018 | Win/lose count 2.5/2.0 (0.5)
Epoch 005/021 | Loss 0.0018 | Win/lose count 5.0/3.0 (2.0)
Epoch 006/021 | Loss 0.0029 | Win/lose count 5.0/5.0 (0.0)
Epoch 007/021 | Loss 0.0048 | Win/lose count 7.0/1.0 (6.0)
Epoch 008/021 | Loss 0.0021 | Win/lose count 4.5/1.0 (3.5)
Epoch 009/021 | Loss 0.0020 | Win/lose count 7.0/5.0 (2.0)
Epoch 010/021 | Loss 0.0106 | Win/lose count 4.5/1.0 (3.5)
Epoch 011/021 | Loss 0.0013 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/021 | Loss 0.0574 | Win/lose count 4.0/0 (4.0)
Epoch 013/021 | Loss 0.0087 | Win/lose count 9.5/1.0 (8.5)
Epoch 014/021 | Loss 0.0016 | Win/lose count 10.0/4.0 (6.0)
Epoch 015/021 | Loss 0.0034 | Win/lose count 8.5/2.0 (6.5)
Epoch 016/021 | Loss 0.0040 | Win/lose count 8.0/7.0 (1.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 4.0/4.0. Average score (0.0)
Win/lose count 5.0/4.0. Average score (0.5)
Win/lose count 3.0/5.0. Average score (-0.3333333333333333)
Win/lose count 6.5/3.0. Average score (0.625)
Win/lose count 6.0/7.0. Average score (0.3)
Win/lose count 4.5/8.0. Average score (-0.3333333333333333)
Win/lose count 4.0/5.0. Average score (-0.42857142857142855)
Win/lose count 4.0/6.0. Average score (-0.625)
Win/lose count 2.5/6.0. Average score (-0.9444444444444444)
Win/lose count 6.5/5.0. Average score (-0.7)
Win/lose count 4.0/5.0. Average score (-0.7272727272727273)
Win/lose count 2.5/3.0. Average score (-0.7083333333333334)
Win/lose count 6.0/0. Average score (-0.19230769230769232)
Win/lose count 3.5/2.0. Average score (-0.07142857142857142)
Win/lose count 3.0/0. Average score (0.13333333333333333)
Final score: 0.13333333333333333
Test of the FC
Win/lose count 5.0/4.0. Average score (1.0)
Win/lose count 2.0/4.0. Average score (-0.5)
Win/lose count 3.5/10.0. Average score

In [21]:
HTML(display_videos('cnn_test10.mp4'))

In [22]:
HTML(display_videos('fc_test10.mp4'))

_________________
__Answer 9:__
- The main issue we can observe is that the rat keeps being stuck between 2 to 4 positions. So in the cases the score is positive, it's just a matter of luck because the rat starts in a good position. We tackled this issue by changing $\varepsilon$ but this was not as optimal as we expected (epsilon is manualy tuned).
- The score increases with the temperature as the temperature gives the proportion of positive/negative rewards on the board.
- The CNN works generally better than the FC net.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
## use those samples of code:


In [0]:
## As we tried before in answer 10 to tune epsilon manualy, we are focusing here
## on this again.
## In the train_explore function, we are going to initalize the model with a
## relatively big value for epsilon and then we have an adjusting factor to decrease
## value. So the rat we explore in the begenning and then doing more exploitation than exploration
## I set a minimum value for epsilon so the model doesn't definitely stop exploring at a certain step

def train_explore(agent,env,epoch,prefix='',epsilon_adjusting_factor=1, min_epsilon=0.1):
    # Number of won games
    score = 0
    loss = 0
    initial_epsilon = agent.epsilon

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(initial_epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            agent.epsilon = max(agent.epsilon*epsilon_adjusting_factor, min_epsilon)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        self.malus_position = np.zeros((grid_size,grid_size))  ##New representation of the grid

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1

        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1  ## There was an error here, we should 
                                                    ## make the value increase instead of setting it at 0.1 every time.
                                                    ## Hence it is += 0.1 instead of = 0.1
                                                    ## This is because we want the agent to consider the malus much bigger in
                                                    ## area it had already visited

        reward = reward + self.board[self.x, self.y]
        

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [29]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore', epsilon_adjusting_factor=0.99, min_epsilon=0.001)
HTML(display_videos('cnn_train_explore20.mp4'))

Epoch 000/025 | Loss 0.0292 | Win/lose count 10.5/84.4 (-73.9)
Epoch 001/025 | Loss 0.0204 | Win/lose count 14.0/29.60000000000002 (-15.60000000000002)
Epoch 002/025 | Loss 0.0127 | Win/lose count 14.5/21.500000000000004 (-7.0000000000000036)
Epoch 003/025 | Loss 0.0180 | Win/lose count 18.5/26.80000000000001 (-8.300000000000011)
Epoch 004/025 | Loss 0.0210 | Win/lose count 21.0/20.700000000000003 (0.29999999999999716)
Epoch 005/025 | Loss 0.0345 | Win/lose count 17.5/25.000000000000025 (-7.500000000000025)
Epoch 006/025 | Loss 0.0872 | Win/lose count 26.5/13.999999999999986 (12.500000000000014)
Epoch 007/025 | Loss 0.0095 | Win/lose count 19.5/17.699999999999992 (1.8000000000000078)
Epoch 008/025 | Loss 0.0136 | Win/lose count 22.5/13.69999999999998 (8.80000000000002)
Epoch 009/025 | Loss 0.0146 | Win/lose count 24.0/18.299999999999994 (5.700000000000006)
Epoch 010/025 | Loss 0.0108 | Win/lose count 22.5/14.099999999999985 (8.400000000000015)
Epoch 011/025 | Loss 0.0231 | Win/lose cou

In [30]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 25.5/9.0. Average score (16.5)
Win/lose count 24.0/2.0. Average score (19.25)
Win/lose count 23.0/5.0. Average score (18.833333333333332)
Win/lose count 23.5/4.0. Average score (19.0)
Win/lose count 22.5/4.0. Average score (18.9)
Win/lose count 22.0/5.0. Average score (18.583333333333332)
Win/lose count 24.5/3.0. Average score (19.0)
Win/lose count 20.0/6.0. Average score (18.375)
Win/lose count 20.5/4.0. Average score (18.166666666666668)
Win/lose count 17.5/2.0. Average score (17.9)
Win/lose count 23.0/1.0. Average score (18.272727272727273)
Win/lose count 28.0/3.0. Average score (18.833333333333332)
Win/lose count 23.0/6.0. Average score (18.692307692307693)
Win/lose count 20.5/5.0. Average score (18.464285714285715)
Win/lose count 23.0/3.0. Average score (18.566666666666666)
Final score: 18.566666666666666


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***